# InfoGAN CeleA

[InfoGAN: Interpretable Representation Learning by Information Maximizing Generative Adversarial Nets](https://arxiv.org/abs/1606.03657).


In [1]:
#Import the libraries we will need.
import os, glob, cv2, math, sys
import tensorflow as tf
import numpy as np
from numpy.random import permutation
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import scipy.misc
import scipy
#from PIL import Image

np.random.seed(1)
#plt.ion()   # interactive mode
%matplotlib inline

load CeleA data

In [2]:
# http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html
#img_rows, img_cols = 100, 100
#img_rows, img_cols = 64, 64
img_rows, img_cols = 32, 32

def get_im(path):

    #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.imread(path)
    #img = plt.imread(path)
    #resized = img
    resized = cv2.resize(img, (img_cols, img_rows))

    return resized

def read_train_data_fullname(path):

    
    files = glob.glob(path)
    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_fullname_lfw(path):

    root = path
    all_folders = os.listdir(root)
    path = []
    for afolder in all_folders:
        path.append(root+"/"+afolder+"/*.jpg")
    #print(path)
    files = []
    for apath in path:
        templist = glob.glob(apath)
        for afile in templist:
            files.append(afile)

    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_batch(filenames, batchsize=5):
    
    end = min(len(filenames), batchsize)
    train_data = []
    
    for fl in filenames[:end]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        
        #normalization
        #img -= np.mean(img)
        #img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    filenames = filenames[end:]
    
    return train_data, filenames

def read_train_data_mini_batch(filenames, startpoint, batchsize=5):
    
    train_data = []
    
    for fl in filenames[startpoint:startpoint+batchsize]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        #print(type(img))        
        #normalization
        img -= np.mean(img)
        img /= np.std(img)
        #print(img.shape)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)
    
    return train_data

#random mini-batch
def read_train_data_random_batch(filenames, batchsize=5):
    fullsize=len(filenames)
    #http://qiita.com/hamukazu/items/ec1b4659df00f0ce43b1
    idset = np.random.randint(0, high=fullsize, size=batchsize)
    #print(idset) 
    train_data = []
    
    for fid in idset:
        fl = filenames[fid]
        flbase = os.path.basename(fl)
        img = get_im(fl)
        #img[:,:,1] = 0
        #img[:,:,2] = 0
        img = np.array(img, dtype=np.float32)
        #img = np.reshape(img,[img.shape[0],img.shape[1],1])
        # normalization
        # https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn
        img -= np.mean(img)
        img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    
    return train_data
# must be full path, ~/... not ok
# train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data,train_data_filenames = read_train_data_batch(train_data_filenames)

#random mini-batch
#train_data = read_train_data_random_batch(train_data_filenames)

In [3]:
#print(len(train_data),len(train_data_filenames))
#print(len(train_data[0][1]))

def imshow(inp, title=None):
    """Imshow for Tensor."""
    # need this transpose if there is transpose in read_train_data_xx calls
    #inp = inp.numpy().transpose((1, 2, 0))
    #inp = std * inp + mean
    #plt.imshow(inp,cmap="Purples",interpolation = 'bicubic')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.01)  # pause a bit so that plots are updated

#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
#train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")
#print(len(train_data_filenames))
#print(train_data_filenames[3])
#train_data = read_train_data_mini_batch(train_data_filenames,0)  
#print(train_data[0].shape)
#print(train_data[43][:,:,0:2].shape)

#imshow(train_data[2][:,:,0])
#imshow(train_data[2][:,:,0])

### Helper Functions

In [4]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge_color(images, size))
    #return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    #return images
    # https://zhuanlan.zhihu.com/p/25051313
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

def merge_color(images, size):
    h, w, c = images.shape[1], images.shape[2],images.shape[3]
    img = np.zeros((h * size[0], w * size[1],c))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image[:,:,:]

    return img


## Defining the Adversarial Networks

### Generator Network

The generator takes a vector of random numbers and transforms it into a 32x32 image. Each layer in the network involves a strided  transpose convolution, batch normalization, and rectified nonlinearity. Tensorflow's slim library allows us to easily define each of these layers.

In [5]:
# https://www.tensorflow.org/api_docs/python/tf/depth_to_space
# http://qiita.com/tadOne/items/48302a399dcad44c69c8   Tensorflow - padding = VALID/SAMEの違いについて
#     so 3 tf.depth_to_space(genX,2) gives 4x2^3 = 32
# 


"""
def generator(z):
    
    zP = slim.fully_connected(z,4*4*448,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,448])
    
    gen1 = slim.convolution2d(\
        zCon,num_outputs=256,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    gen1 = tf.depth_to_space(gen1,2)
    
    gen2 = slim.convolution2d(\
        gen1,num_outputs=128,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    gen2 = tf.depth_to_space(gen2,2)
    
    gen3 = slim.convolution2d(\
        gen2,num_outputs=64,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    gen3 = tf.depth_to_space(gen3,2)
    
    g_out = slim.convolution2d(\
        gen3,num_outputs=3,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out
"""

# use slim.convolution2d_transpose
# http://lib.csdn.net/snippet/deeplearning/67478?knId=1757
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/layers/python/layers/layers.py
# https://www.tensorflow.org/api_docs/python/tf/nn/conv2d_transpose
# https://www.tensorflow.org/api_docs/python/tf/nn/convolution
#      If padding == "SAME": output_spatial_shape[i] = ceil(input_spatial_shape[i] / strides[i])
# so, use padding and stride is enough to control output size. 
# in conv2d, stide=2, padding = same, inputsize =4, output=4/2=2
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/conv2d_transpose
#  https://www.tensorflow.org/versions/r1.3/api_docs/python/tf/contrib/keras/layers/Conv2DTranspose
def generator(z):
    
    zP = slim.fully_connected(z,2*2*448,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,2,2,448])
    
    gen1 = slim.convolution2d_transpose(\
                                       zCon, num_outputs=256, kernel_size=[4,4], stride=[2,2], \
                                       padding="SAME", normalizer_fn=slim.batch_norm,\
                                       activation_fn=tf.nn.relu, scope="g_conv1", weights_initializer=initializer)
    
    #print("gen1",gen1.shape) 
    # result: in convolution2d_transpose padding=same, outputsize = input*stride
    gen2 = slim.convolution2d_transpose(\
                                       gen1, num_outputs=128, kernel_size=[4,4], stride=[2,2], \
                                       padding="SAME", normalizer_fn=slim.batch_norm,\
                                       activation_fn=tf.nn.relu, scope="g_conv2", weights_initializer=initializer)
    
    #print("gen2",gen2.shape) 
    gen3 = slim.convolution2d_transpose(\
                                       gen2, num_outputs=64, kernel_size=[4,4], stride=[2,2], \
                                       padding="SAME", normalizer_fn=slim.batch_norm,\
                                       activation_fn=tf.nn.relu, scope="g_conv3", weights_initializer=initializer)
    
    #print("gen3",gen3.shape) 
    g_out = slim.convolution2d_transpose(\
                                       gen3, num_outputs=3, kernel_size=[4,4], stride=[2,2], \
                                       padding="SAME",biases_initializer=None, \
                                       activation_fn=tf.nn.tanh, scope="g_out", weights_initializer=initializer)
    #print("q_out",g_out.shape)
    return g_out

### Discriminator Network
The discriminator network takes as input a 32x32 image and transforms it into a single valued probability of being generated from real-world data. Again we use tf.slim to define the convolutional layers, batch normalization, and weight initialization.

In [6]:
"""
def discriminator(bottom, cat_list,conts, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    dis1 = tf.space_to_depth(dis1,2)
    
    dis2 = slim.convolution2d(dis1,128,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis2 = tf.space_to_depth(dis2,2)
    
    dis3 = slim.convolution2d(dis2,256,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis3 = tf.space_to_depth(dis3,2)
        
    dis4 = slim.fully_connected(slim.flatten(dis3),1024,activation_fn=lrelu,\
        reuse=reuse,scope='d_fc1', weights_initializer=initializer)
        
    d_out = slim.fully_connected(dis4,1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    q_a = slim.fully_connected(dis4,128,normalizer_fn=slim.batch_norm,\
        reuse=reuse,scope='q_fc1', weights_initializer=initializer)
    
    
    ## Here we define the unique layers used for the q-network. The number of outputs depends on the number of 
    ## latent variables we choose to define.
    q_cat_outs = []
    for idx,var in enumerate(cat_list):
        q_outA = slim.fully_connected(q_a,var,activation_fn=tf.nn.softmax,\
            reuse=reuse,scope='q_out_cat_'+str(idx), weights_initializer=initializer)
        q_cat_outs.append(q_outA)
    
    q_cont_outs = None
    if conts > 0:
        q_cont_outs = slim.fully_connected(q_a,conts,activation_fn=tf.nn.tanh,\
            reuse=reuse,scope='q_out_cont_'+str(conts), weights_initializer=initializer)
    
    #print("d_out"+str(d_out))
    return d_out,q_cat_outs,q_cont_outs
"""

def discriminator(bottom, cat_list,conts, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    
    dis2 = slim.convolution2d(dis1,128,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    
    dis3 = slim.convolution2d(dis2,256,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    
    #print("dis3",dis3.shape)
        
    dis4 = slim.fully_connected(slim.flatten(dis3),1024,activation_fn=lrelu,\
        reuse=reuse,scope='d_fc1', weights_initializer=initializer)
        
    d_out = slim.fully_connected(dis4,1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    q_a = slim.fully_connected(dis4,128,normalizer_fn=slim.batch_norm,\
        reuse=reuse,scope='q_fc1', weights_initializer=initializer)
    
    
    ## Here we define the unique layers used for the q-network. The number of outputs depends on the number of 
    ## latent variables we choose to define.
    q_cat_outs = []
    for idx,var in enumerate(cat_list):
        q_outA = slim.fully_connected(q_a,var,activation_fn=tf.nn.softmax,\
            reuse=reuse,scope='q_out_cat_'+str(idx), weights_initializer=initializer)
        q_cat_outs.append(q_outA)
    
    q_cont_outs = None
    if conts > 0:
        q_cont_outs = slim.fully_connected(q_a,conts,activation_fn=tf.nn.tanh,\
            reuse=reuse,scope='q_out_cont_'+str(conts), weights_initializer=initializer)
    
    #print("d_out"+str(d_out))
    return d_out,q_cat_outs,q_cont_outs

### Connecting them together

In [7]:
# https://www.tensorflow.org/api_docs/python/tf/split
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://www.tensorflow.org/api_docs/python/tf/concat
# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
# https://www.tensorflow.org/api_docs/python/tf/reduce_mean
# https://www.tensorflow.org/api_docs/python/tf/trainable_variables
# https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
# https://deepage.net/deep_learning/2016/10/26/batch_normalization.html
# z_lat: one_hot_size + z_size + number_continuous = 10+64+2=76
# g_loss def is interesting, my understanding: 
#        if Dg is the probablity to be told as feak data, then 1-Dg is the probabily of suceessfully cheating, 
#        so we cal KL(Dg/(1-Dg)), and readuce_mean works as sampling proceduce
# 

tf.reset_default_graph()


z_size = 128 #Size of initial z vector used for generator.

# Define latent variables.
#categorical_list = [10]*10 # Each entry in this list defines a categorical variable of a specific size.
categorical_list = [10] # Each entry in this list defines a categorical variable of a specific size.
# categorical_list = [10,10] # Each entry in this list defines a categorical variable of a specific size.
number_continuous = 1 # The number of continous variables.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,3],dtype=tf.float32) #Real images

#These placeholders load the latent variables.
latent_cat_in = tf.placeholder(shape=[None,len(categorical_list)],dtype=tf.int32)
#print("latent_cat_in:", latent_cat_in)
latent_cat_list = tf.split(latent_cat_in,len(categorical_list),1)
#print("latent_cat_list: ",latent_cat_list)
if number_continuous>0:
    latent_cont_in = tf.placeholder(shape=[None,number_continuous],dtype=tf.float32)

oh_list = []
for idx,var in enumerate(categorical_list):
    latent_oh = tf.one_hot(tf.reshape(latent_cat_list[idx],[-1]),var)
    #print(latent_cat_list[idx])
    #print(latent_oh),  woundn't print anything in sess.run()
    oh_list.append(latent_oh)

#Concatenate all c and z variables.
z_lats = oh_list[:]
#print("1st z_lats: ", z_lats )
z_lats.append(z_in)
#print("2nd z_lats: ", z_lats )
if number_continuous>0:
    z_lats.append(latent_cont_in)
#print("3rd z_lats: ", z_lats )
z_lat = tf.concat(z_lats,1)
#print("z_lat: ", z_lat )

Gz = generator(z_lat) #Generates images from random z vectors
#print ("Gz",Gz.shape)
Dx,_,_ = discriminator(real_in,categorical_list,number_continuous) #Produces probabilities for real images
Dg,QgCat,QgCont = discriminator(Gz,categorical_list,number_continuous,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
#g_loss = -tf.reduce_mean(tf.log((Dg/(1.-Dg)))) #KL Divergence optimizer
g_loss = -tf.reduce_mean(tf.log(Dg)) 

#Combine losses for each of the categorical variables.
cat_losses = []
for idx,latent_var in enumerate(oh_list):
    #print ("latent_var: ", latent_var)
    #print ("tf.log(QgCat[idx]): ",tf.log(QgCat[idx]))
    cat_loss = -tf.reduce_sum(latent_var*tf.log(QgCat[idx]),axis=1)
    cat_losses.append(cat_loss)
    
#Combine losses for each of the continous variables.
if number_continuous > 0:
    q_cont_loss = tf.reduce_sum(0.5 * tf.square(latent_cont_in - QgCont),axis=1)
else:
    q_cont_loss = tf.constant(0.0)

q_cont_loss = tf.reduce_mean(q_cont_loss)
q_cat_loss = tf.reduce_mean(cat_losses)
q_loss = tf.add(q_cat_loss,q_cont_loss)
tvars = tf.trainable_variables()
#print (len(tvars))
#for i in tvars:
#    print(i)

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.4)
trainerQ = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
#
d_grads = trainerD.compute_gradients(d_loss,tvars[9:-2-((number_continuous>0)*2)-(len(categorical_list)*2)]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss, tvars[0:9]) #Only update the weights for the generator network.
q_grads = trainerQ.compute_gradients(q_loss, tvars) 

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)
update_Q = trainerQ.apply_gradients(q_grads)

Gz (?, 32, 32, 3)


## Training the network
Now that we have fully defined our network, it is time to train it!

In [8]:
# on at52 (GTX1080), 15mins/10000 epochs , 5000000 is about 12.5 hrs　 
# https://stackoverflow.com/questions/19349410/how-to-pad-with-zeros-a-tensor-along-some-axis-python
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
# blow up after 81800
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html
# https://www.tensorflow.org/api_docs/python/tf/Session#run
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html
c_val = 10

batch_size = 64 #Size of image batch to apply at each iteration.
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")

#iterations = 500000 #Total number of iterations to use.
iterations = 1000 #Total number of iterations to use.
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to save trained model to.

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    for i in range(iterations):
        train_data_filenames=permutation(train_data_filenames) # mini-batch
        data_left = len(train_data_filenames)
        batch_counter = 0
        while data_left>0:
            batch_size_to_train = min(batch_size, data_left)          
            
            zs = np.random.uniform(-1.0,1.0,size=[batch_size_to_train,z_size]).astype(np.float32) #Generate a random z batch
            #print("zs shape:",zs.shape)
            
            #lcat = np.random.randint(0,10,[batch_size,len(categorical_list)]) #Generate random c batch
            lcat = np.random.randint(0,c_val,[batch_size_to_train,len(categorical_list)]) #Generate random c batch
            
            lcont = np.random.uniform(-1,1,[batch_size_to_train,number_continuous]) #

            #xs = read_train_data_random_batch(train_data_filenames, batchsize=batch_size_to_train)
            xs = read_train_data_mini_batch(train_data_filenames, batch_counter*batch_size, batch_size_to_train)
            
            _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the discriminator
            _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the generator, twice for good measure.
            _,qLoss,qK,qC = sess.run([update_Q,q_loss,q_cont_loss,q_cat_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update to optimize mutual information.

            data_left = data_left - batch_size_to_train
            batch_counter +=1
            if batch_counter%1 == 0 or data_left == 0:
                z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32)
                lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
                a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
                b = np.reshape(a,[c_val*c_val,1])
                lcont_sample = b
                samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample})
                if not os.path.exists(sample_directory):
                    os.makedirs(sample_directory)
                save_images(np.reshape(samples[0:100],[100,32,32,3]),[10,10],sample_directory+'/fig'\
                            +str(i)+'_'+str(batch_counter)+'.png')
                
                print ("epoch:"+str(i)+" batch_done:"+str(batch_counter) \
                       +" Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            
             
        """
        if i % 100 == 0:
            print ("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
            z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
            #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
            lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
            latent_fixed = np.ones((c_val*c_val,1))
            lcat_sample = np.hstack([latent_fixed,lcat_sample])
            
            #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
            a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
            #b = np.reshape(a,[100,1])
            b = np.reshape(a,[c_val*c_val,1])
            c = np.zeros_like(b)
            lcont_sample = np.hstack([b,c])
            #
            samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig'+str(i)+'.png')
            save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig'+str(i)+'.png')
        """
        
        if i % 10 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print ("Saved Model on ", i)


epoch:0 batch_done:1 Gen Loss: 2.17071 Disc Loss: 1.32554 Q Losses: [0.19405454, 2.3524222]
epoch:0 batch_done:2 Gen Loss: 2.451 Disc Loss: 1.18583 Q Losses: [0.15907905, 2.2871699]
epoch:0 batch_done:3 Gen Loss: 2.87966 Disc Loss: 1.10141 Q Losses: [0.15069036, 2.2845907]
epoch:0 batch_done:4 Gen Loss: 2.98231 Disc Loss: 0.99932 Q Losses: [0.14914382, 2.2959218]
epoch:0 batch_done:5 Gen Loss: 3.24583 Disc Loss: 1.04598 Q Losses: [0.12683144, 2.2750087]
epoch:0 batch_done:6 Gen Loss: 3.60886 Disc Loss: 1.13703 Q Losses: [0.12395483, 2.2537293]
epoch:0 batch_done:7 Gen Loss: 4.21808 Disc Loss: 0.887093 Q Losses: [0.11323531, 2.2702179]
epoch:0 batch_done:8 Gen Loss: 4.24637 Disc Loss: 0.804176 Q Losses: [0.11007988, 2.2781672]
epoch:0 batch_done:9 Gen Loss: 4.78517 Disc Loss: 0.715185 Q Losses: [0.11622417, 2.2460232]
epoch:0 batch_done:10 Gen Loss: 5.59235 Disc Loss: 0.587384 Q Losses: [0.14083552, 2.2328839]
epoch:0 batch_done:11 Gen Loss: 5.96066 Disc Loss: 0.670109 Q Losses: [0.1355

epoch:0 batch_done:90 Gen Loss: 5.87994 Disc Loss: 0.207948 Q Losses: [0.044272717, 1.0395212]
epoch:0 batch_done:91 Gen Loss: 7.07961 Disc Loss: 0.326388 Q Losses: [0.04724564, 1.0286057]
epoch:0 batch_done:92 Gen Loss: 6.8191 Disc Loss: 0.0975288 Q Losses: [0.055689558, 0.99066889]
epoch:0 batch_done:93 Gen Loss: 5.44382 Disc Loss: 0.16094 Q Losses: [0.051049281, 0.93952054]
epoch:0 batch_done:94 Gen Loss: 7.82316 Disc Loss: 0.385974 Q Losses: [0.053722814, 0.9322356]
epoch:0 batch_done:95 Gen Loss: 5.92353 Disc Loss: 0.347308 Q Losses: [0.040749654, 0.93804723]
epoch:0 batch_done:96 Gen Loss: 7.21013 Disc Loss: 0.431963 Q Losses: [0.062268578, 0.91799784]
epoch:0 batch_done:97 Gen Loss: 6.75364 Disc Loss: 0.171078 Q Losses: [0.0517218, 0.90126431]
epoch:0 batch_done:98 Gen Loss: 6.96373 Disc Loss: 0.156985 Q Losses: [0.056429133, 0.89858723]
epoch:0 batch_done:99 Gen Loss: 7.55771 Disc Loss: 0.252176 Q Losses: [0.055679236, 0.89621198]
epoch:0 batch_done:100 Gen Loss: 7.36284 Disc L

epoch:0 batch_done:175 Gen Loss: 8.89704 Disc Loss: 0.187895 Q Losses: [0.040717646, 0.41243827]
epoch:0 batch_done:176 Gen Loss: 7.09303 Disc Loss: 0.122147 Q Losses: [0.045954827, 0.41991937]
epoch:0 batch_done:177 Gen Loss: 4.89544 Disc Loss: 0.0895585 Q Losses: [0.038493425, 0.4146271]
epoch:0 batch_done:178 Gen Loss: 11.9803 Disc Loss: 0.182051 Q Losses: [0.040493257, 0.42350394]
epoch:0 batch_done:179 Gen Loss: 8.62206 Disc Loss: 0.268108 Q Losses: [0.042924415, 0.42334124]
epoch:0 batch_done:180 Gen Loss: 5.09056 Disc Loss: 0.0211176 Q Losses: [0.037744537, 0.40540445]
epoch:0 batch_done:181 Gen Loss: 15.0419 Disc Loss: 0.207899 Q Losses: [0.042298205, 0.40844887]
epoch:0 batch_done:182 Gen Loss: 6.66759 Disc Loss: 0.643243 Q Losses: [0.036362879, 0.40571809]
epoch:0 batch_done:183 Gen Loss: 5.44595 Disc Loss: 0.0838659 Q Losses: [0.024607019, 0.39114544]
epoch:0 batch_done:184 Gen Loss: 26.7738 Disc Loss: 0.736348 Q Losses: [0.04774962, 0.45814812]
epoch:0 batch_done:185 Gen Lo

epoch:1 batch_done:55 Gen Loss: 6.41381 Disc Loss: 0.0411889 Q Losses: [0.020700427, 0.25164962]
epoch:1 batch_done:56 Gen Loss: 24.7164 Disc Loss: 2.99766 Q Losses: [0.023289867, 0.26588467]
epoch:1 batch_done:57 Gen Loss: 28.0666 Disc Loss: 0.258067 Q Losses: [0.024844497, 0.26697773]
epoch:1 batch_done:58 Gen Loss: 22.9162 Disc Loss: 2.24059 Q Losses: [0.037226602, 0.23876496]
epoch:1 batch_done:59 Gen Loss: 17.2298 Disc Loss: 0.293551 Q Losses: [0.029476266, 0.23774669]
epoch:1 batch_done:60 Gen Loss: 11.7642 Disc Loss: 0.00662477 Q Losses: [0.01811536, 0.24072701]
epoch:1 batch_done:61 Gen Loss: 6.60927 Disc Loss: 0.00218889 Q Losses: [0.021190103, 0.21104136]
epoch:1 batch_done:62 Gen Loss: 5.88526 Disc Loss: 0.142753 Q Losses: [0.029525388, 0.21529156]
epoch:1 batch_done:63 Gen Loss: 8.45846 Disc Loss: 0.160237 Q Losses: [0.033119794, 0.22126344]
epoch:1 batch_done:64 Gen Loss: 7.77814 Disc Loss: 0.0921622 Q Losses: [0.023367126, 0.20950373]
epoch:1 batch_done:65 Gen Loss: 6.431

epoch:1 batch_done:142 Gen Loss: 22.1341 Disc Loss: 1.52607 Q Losses: [0.016386889, 0.14069481]
epoch:1 batch_done:143 Gen Loss: 19.3105 Disc Loss: 0.224868 Q Losses: [0.016508281, 0.14668688]
epoch:1 batch_done:144 Gen Loss: 15.6324 Disc Loss: 0.0313943 Q Losses: [0.01766878, 0.15682714]
epoch:1 batch_done:145 Gen Loss: 11.2647 Disc Loss: 0.0200229 Q Losses: [0.027296856, 0.13226128]
epoch:1 batch_done:146 Gen Loss: 6.03856 Disc Loss: 0.00519621 Q Losses: [0.024602551, 0.13628072]
epoch:1 batch_done:147 Gen Loss: 11.9331 Disc Loss: 0.274689 Q Losses: [0.021344665, 0.12468629]
epoch:1 batch_done:148 Gen Loss: 12.271 Disc Loss: 0.116809 Q Losses: [0.021130834, 0.13600372]
epoch:1 batch_done:149 Gen Loss: 9.26073 Disc Loss: 0.175552 Q Losses: [0.02631221, 0.15346368]
epoch:1 batch_done:150 Gen Loss: 3.92372 Disc Loss: 0.288309 Q Losses: [0.015227243, 0.1405172]
epoch:1 batch_done:151 Gen Loss: 22.2782 Disc Loss: 0.868878 Q Losses: [0.021234695, 0.13000241]
epoch:1 batch_done:152 Gen Loss

...

epoch:77 batch_done:200 Gen Loss: nan Disc Loss: nan Q Losses: [0.19874749, 2.3023918]
epoch:77 batch_done:201 Gen Loss: nan Disc Loss: nan Q Losses: [0.19348697, 2.3027782]
epoch:77 batch_done:202 Gen Loss: nan Disc Loss: nan Q Losses: [0.1708478, 2.3027177]
epoch:77 batch_done:203 Gen Loss: nan Disc Loss: nan Q Losses: [0.1455102, 2.3023846]
epoch:77 batch_done:204 Gen Loss: nan Disc Loss: nan Q Losses: [0.17172903, 2.301158]
epoch:77 batch_done:205 Gen Loss: nan Disc Loss: nan Q Losses: [0.16232747, 2.3029211]
epoch:77 batch_done:206 Gen Loss: nan Disc Loss: nan Q Losses: [0.14791882, 2.3017976]
epoch:77 batch_done:207 Gen Loss: nan Disc Loss: nan Q Losses: [0.16637048, 2.301888]
epoch:78 batch_done:1 Gen Loss: nan Disc Loss: nan Q Losses: [0.17620967, 2.3016357]
epoch:78 batch_done:2 Gen Loss: nan Disc Loss: nan Q Losses: [0.12808096, 2.3039839]
epoch:78 batch_done:3 Gen Loss: nan Disc Loss: nan Q Losses: [0.12972406, 2.3022914]
epoch:78 batch_done:4 Gen Loss: nan Disc Loss: nan Q 

epoch:78 batch_done:89 Gen Loss: nan Disc Loss: nan Q Losses: [0.18444711, 2.3025422]
epoch:78 batch_done:90 Gen Loss: nan Disc Loss: nan Q Losses: [0.16594678, 2.3029561]
epoch:78 batch_done:91 Gen Loss: nan Disc Loss: nan Q Losses: [0.1438746, 2.3022294]
epoch:78 batch_done:92 Gen Loss: nan Disc Loss: nan Q Losses: [0.16919123, 2.3029721]
epoch:78 batch_done:93 Gen Loss: nan Disc Loss: nan Q Losses: [0.18719013, 2.3034286]
epoch:78 batch_done:94 Gen Loss: nan Disc Loss: nan Q Losses: [0.1953108, 2.3027492]
epoch:78 batch_done:95 Gen Loss: nan Disc Loss: nan Q Losses: [0.16112277, 2.3020072]
epoch:78 batch_done:96 Gen Loss: nan Disc Loss: nan Q Losses: [0.18339868, 2.3024988]
epoch:78 batch_done:97 Gen Loss: nan Disc Loss: nan Q Losses: [0.18744254, 2.30143]
epoch:78 batch_done:98 Gen Loss: nan Disc Loss: nan Q Losses: [0.16709842, 2.3040895]
epoch:78 batch_done:99 Gen Loss: nan Disc Loss: nan Q Losses: [0.17326027, 2.3019164]
epoch:78 batch_done:100 Gen Loss: nan Disc Loss: nan Q Los

epoch:78 batch_done:185 Gen Loss: nan Disc Loss: nan Q Losses: [0.16228594, 2.3015995]
epoch:78 batch_done:186 Gen Loss: nan Disc Loss: nan Q Losses: [0.17151681, 2.3022685]
epoch:78 batch_done:187 Gen Loss: nan Disc Loss: nan Q Losses: [0.12536019, 2.3021762]
epoch:78 batch_done:188 Gen Loss: nan Disc Loss: nan Q Losses: [0.14990854, 2.3015738]
epoch:78 batch_done:189 Gen Loss: nan Disc Loss: nan Q Losses: [0.20007782, 2.3023429]
epoch:78 batch_done:190 Gen Loss: nan Disc Loss: nan Q Losses: [0.16329339, 2.3008449]
epoch:78 batch_done:191 Gen Loss: nan Disc Loss: nan Q Losses: [0.17252538, 2.3030148]
epoch:78 batch_done:192 Gen Loss: nan Disc Loss: nan Q Losses: [0.15736969, 2.3021758]
epoch:78 batch_done:193 Gen Loss: nan Disc Loss: nan Q Losses: [0.18601134, 2.30266]
epoch:78 batch_done:194 Gen Loss: nan Disc Loss: nan Q Losses: [0.16317743, 2.302289]
epoch:78 batch_done:195 Gen Loss: nan Disc Loss: nan Q Losses: [0.16594982, 2.3028774]
epoch:78 batch_done:196 Gen Loss: nan Disc Los

epoch:79 batch_done:75 Gen Loss: nan Disc Loss: nan Q Losses: [0.1615321, 2.3026567]
epoch:79 batch_done:76 Gen Loss: nan Disc Loss: nan Q Losses: [0.15424031, 2.3023958]
epoch:79 batch_done:77 Gen Loss: nan Disc Loss: nan Q Losses: [0.1949943, 2.3045676]
epoch:79 batch_done:78 Gen Loss: nan Disc Loss: nan Q Losses: [0.17029604, 2.302959]
epoch:79 batch_done:79 Gen Loss: nan Disc Loss: nan Q Losses: [0.17565416, 2.3023381]
epoch:79 batch_done:80 Gen Loss: nan Disc Loss: nan Q Losses: [0.15334716, 2.3020964]
epoch:79 batch_done:81 Gen Loss: nan Disc Loss: nan Q Losses: [0.14795159, 2.3028142]
epoch:79 batch_done:82 Gen Loss: nan Disc Loss: nan Q Losses: [0.19102272, 2.3039241]
epoch:79 batch_done:83 Gen Loss: nan Disc Loss: nan Q Losses: [0.16344413, 2.301465]
epoch:79 batch_done:84 Gen Loss: nan Disc Loss: nan Q Losses: [0.18823658, 2.3026707]
epoch:79 batch_done:85 Gen Loss: nan Disc Loss: nan Q Losses: [0.17888847, 2.3026943]
epoch:79 batch_done:86 Gen Loss: nan Disc Loss: nan Q Loss

KeyboardInterrupt: 

## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [ ]:
# http://qiita.com/TokyoMickey/items/f6a9251f5a59120e39f8
"""
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.

#init = tf.initialize_all_variables()
c_val = 10

saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    #sess.run(init)
    #Reload the model.
    print ('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
    z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
    #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
    #lcat_sample = np.reshape(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]),[100,1])
    lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
    #print(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]))
    #latent_fixed = np.ones((c_val*c_val,1))*50
    latent_fixed = np.zeros((c_val*c_val,1))
    #lcat_sample = np.hstack([latent_fixed,lcat_sample])
    # good shape
    lcat_sample = np.hstack([lcat_sample,latent_fixed])
            
    #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
    a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #a = a = np.ones((c_val*c_val,1))*-0.5
    #a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #b = np.reshape(a,[100,1])
    b = np.reshape(a,[c_val*c_val,1])
    #c = np.zeros_like(b)
    #c = np.zeros_like(b)
    c = np.zeros_like(b)
    #c = np.zeros_like(b)+8
    #angle
    lcont_sample = np.hstack([b,c])
    # width
    #lcont_sample = np.hstack([c,b])
    
    samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    #Save sample generator images for viewing training progress.
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test'+'.png')
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test_4'+'.png')
    save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig_test_13'+'.png')
""" 